#### Note: This will run if you have the the movies saved as "start_numor".npz files. Check other notebook titled SANS_to_npz.ipynb for code to assist in doing this.

In [ ]:
"""
to run the notebook, first replace all file paths (check for `np.load` and `file_path`).
search for `gif = movies[i]` where this line allows you to change the set of data
Parameters to tune: `FILTER_SIGNAL_THRESHOLD`, `laser_threshold`, `angle_above_offset` (in fix snapback section where you can decide if you want to change offsets)
"""

# Gradient Descent (2 filters)

#### functions

In [ ]:
import numpy as np
from PIL import Image, ImageSequence
import matplotlib.pyplot as plt
import torch
from scipy.ndimage import gaussian_filter
from matplotlib.animation import FuncAnimation


In [ ]:
# Replace with your file path
file_path = '/Users/cadenmyers/billingelab/dev/sym_adapted_filts/experimental_data/46349.npz'
data = np.load(file_path)['data']

im_shape = data[0].shape
# im_shape = (128, 128)
x, y = np.meshgrid(
    np.arange(-im_shape[0] // 2, im_shape[0] // 2),
    np.arange(-im_shape[1] // 2, im_shape[1] // 2),
)
DATA_THETA = torch.atan2(torch.tensor(x), torch.tensor(y))
MAX_ITER_OFFSET = 51
LR = 1e-2

n_folds = 6
k=8.4
print("n_folds =", n_folds)
print('k value =', k)
def filter_function(k, theta, n_folds=n_folds):
    filter = torch.exp(k * torch.log((torch.cos(n_folds / 2 * theta))**2))
    # plt.imshow(filter)
    # plt.title(f'n_folds={n_folds}, k={k}')
    # plt.show()
    return filter

def normalize_min_max(data):
    if isinstance(data, torch.Tensor):
        array = data.numpy()
    else:
        array = data

    array_min = np.min(array)
    array_max = np.max(array)
    norm_array = (array - array_min) / (array_max - array_min)

    if isinstance(data, torch.Tensor):
        norm_tensor = torch.tensor(norm_array)
        return norm_tensor
    else:
        return norm_array

def mask_and_blur_images(array, sigma=1):
    '''Masks signal inside radius of 14 and outside radius of 30 and adds gaussian blur for all intensity data'''
    x, y = np.meshgrid(np.arange(128), np.arange(128))
    radius = np.sqrt((x - 64) ** 2 + (y - 62) ** 2)
    mask1 = radius <= 14
    mask2 = radius >= 30
    masked_data = array.copy()
    masked_data[mask1] = 0
    masked_data2 = masked_data.copy()
    masked_data2[mask2] = 0
    blurred_data = gaussian_filter(masked_data2, sigma=sigma)
    return blurred_data


In [ ]:
# SIMPLE GD FUNCTION

def gradient_descent_optimize_offset(intensity, offset, k=k):
    opt = torch.optim.Adam([offset], lr=LR)
    for i in range(MAX_ITER_OFFSET):
        evaluate_image_theta = filter_function(k, DATA_THETA+offset)
        loss = -(intensity * evaluate_image_theta).sum()
        opt.zero_grad()
        loss.backward()
        opt.step()
    return offset, evaluate_image_theta, loss

# offset, image, loss = gradient_descent_optimize_offset(intensity_data[0], offset1)
# fig, axes = plt.subplots(1, 3, figsize=(15, 5))

# im1 = axes[0].imshow(normalize_min_max(intensity_data[0]) - mask_images(image.detach().numpy()), cmap='viridis')
# axes[0].axis('off')
# fig.colorbar(im1, ax=axes[0])

# im2 = axes[1].imshow(image.detach().numpy(), cmap='viridis')
# axes[1].axis('off')
# fig.colorbar(im2, ax=axes[1])

# im3 = axes[2].imshow(normalize_min_max(intensity_data[0]), cmap='viridis')
# axes[2].axis('off')
# fig.colorbar(im3, ax=axes[2])

# plt.tight_layout()
# plt.show()

In [ ]:
# Helper function to fix snapback
def adjust_offset_within_bounds(offset_list, angle_above_offset=50):
    angle_below_offset = 60 - angle_above_offset
    adjusted_offsets = []
    prev_offset = offset_list[0]
    for index, offset in enumerate(offset_list):
        if index == 0:
            adjusted_offsets.append(offset)
            prev_offset = offset
        else:
            offset_range = (prev_offset - angle_below_offset, prev_offset + angle_above_offset)
            while not (offset_range[0] <= offset <= offset_range[1]):
                offset += OFFSET_ADJUSTMENT if offset < offset_range[0] else -OFFSET_ADJUSTMENT
            adjusted_offsets.append(offset)
            prev_offset = offset
    return adjusted_offsets

In [ ]:
# Global parameters for model (usually we don't need to change this)
MAX_ITER_OFFSET = 101
LR = 1e-2
OFFSET_ADJUSTMENT = 60
FILTER_SIGNAL_THRESHOLD = 0.4

In [ ]:
# TRYING TWO FILTER GD
'''Currently, this code has a penalty function with shape the same as the filter function.
Also, a condition is in place such that if the second loss is less than the 30% of the first loss,
offset2 = offset1.'''

offset1 = torch.tensor(0., requires_grad=True)
offset_second = torch.tensor(0., requires_grad=True)
penalty_strength = 1.0E6

def gradient_descent_optimize_offset(intensity, offset, offset_second, k=k, penalty_strength=penalty_strength):
    # First Gradient Descent to find the global maximum
    opt = torch.optim.Adam([offset], lr=LR)
    for i in range(MAX_ITER_OFFSET):
        evaluate_image_theta = filter_function(k, DATA_THETA + offset)
        loss = -(intensity * evaluate_image_theta).sum()
        opt.zero_grad()
        loss.backward()
        opt.step()

    # Store the first optimal offset
    first_offset = offset.clone().detach()

    # Second Gradient Descent with penalty to avoid first peak
    opt_second = torch.optim.Adam([offset_second], lr=LR)
    for i in range(MAX_ITER_OFFSET):
        evaluate_image_theta_second = filter_function(k, DATA_THETA + offset_second)
        penalty = penalty_strength * filter_function(1/2*k, offset_second - first_offset)

        # Modified loss with penalty
        loss_second = -(intensity * evaluate_image_theta_second).sum() + penalty
        opt_second.zero_grad()
        loss_second.backward()
        opt_second.step()

    # If the second loss is significantly smaller than the first, set offset_second to first_offset
    if np.abs(loss_second.item()) < 0.3 * np.abs(loss.item()):
        offset_second = first_offset.clone().detach()
        evaluate_image_theta_second = filter_function(k, DATA_THETA + offset_second)

    return first_offset, evaluate_image_theta, loss, offset_second, evaluate_image_theta_second, loss_second



#### Apply functions to real data

In [ ]:
# Import images from .npz files
# Extract data file paths
file_path = "/Users/cadenmyers/billingelab/dev/skyrmion_lattices/experimental_data/npz_sept_data/npz_field_sweep/"

# TEMP SWEEP MOVIES
# movies = ['121855.npz', '118923.npz', '119486.npz', '119996.npz', '120506.npz', '121016.npz', '121405.npz', '121550.npz', '122365.npz', '122875.npz']

# FIELD SWEEP MOVIES OLD
#movies = ['Field_29mT.npz', 'Field_31mT.npz', 'Field_32mT.npz', 'Field_33mT.npz', 'Field_37mT.npz']

#SEPT DATA
movies = ['neg23mT_553_50mW.npz', 'neg23mT_558_25mW.npz', 'neg25mT_553_50mW.npz', 'neg25mT_558_25mW.npz', 'neg27mT_553_50mW.npz', 
          'neg27mT_558_25mW.npz', 'neg29mT_553_50mW.npz', 'neg29mT_558_25mW.npz', 'neg31mT_553_50mW.npz', 'neg31mT_558_25mW.npz', 
          'neg33mT_553_50mW.npz', 'neg33mT_558_25mW.npz', 'neg35mT_553_50mW.npz', 'pos23mT_553_50mW.npz', 'pos23mT_558_25mW.npz', 
          'pos25mT_553_50mW.npz', 'pos25mT_558_25mW.npz', 'pos27mT_553_50mW.npz', 'pos27mT_558_25mW.npz', 'pos29mT_553_50mW.npz', 
          'pos29mT_558_25mW.npz', 'pos31mT_553_50mW.npz', 'pos31mT_558_25mW.npz', 'pos33mT_553_50mW.npz', 'pos33mT_558_25mW.npz']

# Define the movie you want to run GD and GS on as gif (gif = movies[i])
#movies = ['pos29mT_558_50mW.npz']
gif = movies[15]
print(gif)

movie = np.load(file_path + gif)
intensity_data = torch.tensor(movie['data'])

# Parameters:
#   iterations: Number of iterations to run the diffusion process.
#   kappa: Threshold for edge stopping (higher means less edge detection).
#   gamma: Step size (controls diffusion speed).
niter=50
kappa=30
gamma=.1
intensity_data = torch.tensor(gaussian_filter(intensity_data, sigma=.4))
print('Tensor shape should be (X,128,128), where X is the number of images.')
print(intensity_data.shape)
plt.imshow(intensity_data[10])

In [ ]:
# GD TWO FILTER
offset_list1 = []
offset_list2 = []
first_image_list = []
second_image_list = []
offset1 = torch.tensor(0.0, requires_grad=True)
offset2 = torch.tensor(0.0, requires_grad=True)  # Ensure this is defined
previous_offset1 = torch.tensor(0.0)  # Track the previous first_offset
k = k_val(resolution)
end_frame = 299

for index, image in enumerate(intensity_data[:end_frame]):
    # Perform gradient descent optimization
    offset1, image1, loss1, offset2, image2, loss2 = gradient_descent_optimize_offset(image, offset1, offset2, k=k, penalty_strength=penalty_strength)
    
    offset1 = torch.tensor(offset1, requires_grad=True)
    offset2 = torch.tensor(offset2, requires_grad=True)
    
    # Check the condition for switching offsets
    # if index > 0 and abs(np.rad2deg(offset1.item()) - np.rad2deg(previous_offset1.item())) >= 10:
    #     offset1, offset2 = offset2, offset1
    #     image1, image2 = image2, image1

    # Append results to the lists
    offset_list1.append(np.rad2deg(offset1.item()))
    offset_list2.append(np.rad2deg(offset2.item()))
    first_image_list.append(image1)
    second_image_list.append(image2)

    # Update previous_offset for the next iteration
    previous_offset1 = offset1.clone()  # Store the current first_offset for the next iteration

    # Print the current offsets
    print(f'{(index + 1) * 10}s: offset 1 = {np.rad2deg(offset1.item())}, offset 2 = {np.rad2deg(offset2.item())}')
    print('GD 1 loss:', loss1.item())
    print('GD 2 loss:', loss2.item())

    # PLOTTING
    fig, axes = plt.subplots(1, 4, figsize=(15, 5))
    axes[0].imshow(image1.detach().T + normalize_min_max(intensity_data[index]).T, origin='lower')
    axes[0].set_title('First GD')
    axes[0].axis('off')
    axes[1].imshow(image2.detach().T + normalize_min_max(intensity_data[index]).T, origin='lower')
    axes[1].set_title('Second GD')
    axes[1].axis('off')
    axes[2].imshow(image1.detach().T + image2.detach().T * 2 + normalize_min_max(intensity_data[index]).T, origin='lower')
    axes[2].set_title('First + Second GD')
    axes[2].axis('off')
    axes[3].imshow(intensity_data[index].T, origin='lower')
    axes[3].set_title('Intensity Data')
    axes[3].axis('off')

    plt.tight_layout()
    plt.show()



In [ ]:
print(offset_list1)

In [ ]:
# MAKE GIF

frame_ranges = np.arange(0,end_frame,1)
# Assuming normalize_min_max, intensity_data, and a way to generate first_image and second_image dynamically are defined
fig, axes = plt.subplots(1, 4, figsize=(15, 5))
# Function to get first_image and second_image based on the current frame index `i`
def get_images(i):
    first_image = first_image_list[i]  # Replace with actual logic
    second_image = second_image_list[i]  # Replace with actual logic
    return first_image, second_image
# Update function for the animation
def update(i):
    first_image, second_image = get_images(i)  # Update images each frame
    for ax in axes:
        ax.clear()
    axes[0].imshow(first_image.detach().T + normalize_min_max(intensity_data[i]).T, origin='lower')
    axes[0].set_title('First GD')
    axes[0].axis('off')
    axes[1].imshow(second_image.detach().T + normalize_min_max(intensity_data[i]).T, origin='lower')
    axes[1].set_title('Second GD')
    axes[1].axis('off')
    axes[2].imshow(first_image.detach().T + second_image.detach().T * 2 + normalize_min_max(intensity_data[i]).T, origin='lower')
    axes[2].set_title('First + Second GD')
    axes[2].axis('off')
    axes[3].imshow(intensity_data[i].T, origin='lower')
    axes[3].set_title(f'{gif}, {(i+1)*10}s')
    axes[3].axis('off')
# Number of frames (change as needed)
frames = len(frame_ranges)
# Create animation
ani = FuncAnimation(fig, update, frames=frames, interval=200)  # Adjust interval for speed (in ms)
# Save as GIF, uncomment to save
ani.save('hexagon_rotation.gif', writer='pillow')
plt.show()

In [ ]:
# GD WITH SUBTRACTING INTENSITIES

# Loop through the movie
# offset_list1, offset_list2 = [], []
# offset1 = torch.tensor(0., requires_grad=True)
# offset2 = torch.tensor(0., requires_grad=True)
# for index, image in enumerate(intensity_data):
#     offset1, offset2, loss1, loss2 = optimize_offset_2filters(image, offset1, offset2)
#     print(f'{(index + 1) * 10}s: offset 1 = {np.rad2deg(offset1.item())}, offset 2 = {np.rad2deg(offset2.item())}')
#     print('GD 1 loss:', loss1.item())
#     print('GD 2 loss:', loss2.item())
#     offset_list1.append(np.rad2deg(offset1.item())), offset_list2.append(np.rad2deg(offset2.item()))

# # Plot offset angles
# time = np.array(range(len(offset_list1))) * 10 + 10
# fig, ax = plt.subplots(nrows=1, ncols=2, figsize=(10, 5))
# ax[0].plot(offset_list1, label="offset1")
# ax[1].plot(offset_list2, label="offset2")
# ax[0].legend()
# ax[1].legend()
# plt.show()

# print(offset_list1)
# print(offset_list2)

In [ ]:
frame_ranges = np.arange(0,end_frame,1)

plt.plot(10*(frame_ranges+1), np.array(offset_list1), label = 'offset 1')
# offset_list2_fix = adjust_offset_within_bounds(offset_list2, 5) 
plt.plot(10*(frame_ranges+1), offset_list2, label = 'offset 2')
plt.xlabel('time (s)')
plt.ylabel('Offset angle')
plt.title(f'{gif}, GD with penalty')
plt.grid(True)
plt.legend()
plt.show()



In [ ]:
# If you want to fix snapback
adjusted_offset_list1 = offset_list1#adjust_offset_within_bounds(offset_list1, angle_above_offset=10)
adjusted_offset_list2 = offset_list2#adjust_offset_within_bounds(offset_list2, angle_above_offset=20)

# Plot offset angles
time = np.array(range(len(offset_list1))) * 10 + 10
fig, ax = plt.subplots(nrows=1, ncols=2, figsize=(10, 5))
ax[0].plot(adjusted_offset_list1, label="adjusted offset1")
ax[1].plot(adjusted_offset_list2, label="adjusted offset2")
ax[0].legend()
ax[1].legend()
plt.show()

# Save model data
adjusted_offset_list1=np.array(adjusted_offset_list1)
adjusted_offset_list2=np.array(adjusted_offset_list2)
# file_path = r'C:\Users\Nathan\OneDrive - nd.edu\Desktop\SANS Data\Experiments\PSI Cu2OSeO3 Corbino July 2023\Analysis\Field Sweep\Peak Tracking npz files\\'
file_path = rf'/Users/yucongchen/billingegroup/skyrmion_lattices/skyrmion-lattices-data/Field_Sweep_data/angles/'
full_path = file_path + gif
np.savez(full_path, gif, offset1=adjusted_offset_list1, offset2=adjusted_offset_list2, time=time)



# Angular velocity calculation

In [ ]:
movies = ['Field_29mT.npz', 'Field_31mT.npz', 'Field_32mT.npz', 'Field_33mT.npz','Field_37mT.npz']

for gif in movies:
    # ratchet_model_data = np.load(rf'C:\Users\Nathan\OneDrive - nd.edu\Desktop\SANS Data\Experiments\PSI Cu2OSeO3 Corbino July 2023\Analysis\Field Sweep\Peak Tracking npz files\{gif}')
    ratchet_model_data = np.load(rf'/Users/yucongchen/billingegroup/skyrmion_lattices/skyrmion-lattices-data/Field_Sweep_data/angles/{gif}')
    rm_time = ratchet_model_data['time']
    rm_offset1 = ratchet_model_data['offset1']
    rm_offset2 = ratchet_model_data['offset2']
    plt.plot(rm_time, rm_offset1, label=f'{gif}, offset1', alpha=.7)
    plt.plot(rm_time, rm_offset2, label=f'{gif}, offset2', alpha=.7)

plt.xlabel('time (s)')
# plt.xlim(0, 380)
plt.ylabel('offset angle (deg)')
plt.grid(True)
plt.legend(loc='upper right', bbox_to_anchor=(1.5, 0.8))
plt.title('Field Sweep offset angle')
# plt.tight_layout()
# plt.savefig(r'C:\Users\Nathan\OneDrive - nd.edu\Desktop\SANS Data\Experiments\PSI Cu2OSeO3 Corbino July 2023\Analysis\Field Sweep\Exported Figures\FieldSweepPositions.png')
plt.show()


In [ ]:
from scipy.signal import savgol_filter
def compute_smoothed_derivative(time, offset, window_length=11, polyorder=2):
    '''compute velocity of data after savgol_filter is applied'''
    smoothed_angle = savgol_filter(offset, window_length=window_length, polyorder=polyorder)
    time = np.array(time)
    smoothed_derivative = (np.gradient(smoothed_angle, time))
    return smoothed_derivative

# calculate angular velo and plot
for gif in movies:
    # ratchet_model_data = np.load(rf'C:\Users\Nathan\OneDrive - nd.edu\Desktop\SANS Data\Experiments\PSI Cu2OSeO3 Corbino July 2023\Analysis\Field Sweep\Peak Tracking npz files\{gif}')
    ratchet_model_data = np.load(rf'/Users/yucongchen/billingegroup/skyrmion_lattices/skyrmion-lattices-data/Field_Sweep_data/angles/{gif}')
    rm_time = ratchet_model_data['time']
    rm_offset1 = ratchet_model_data['offset1']
    rm_offset2 = ratchet_model_data['offset2']
    velo1 = compute_smoothed_derivative(rm_time, rm_offset1)
    velo2 = compute_smoothed_derivative(rm_time, rm_offset2)
    plt.plot(rm_time, velo1, label=f'{gif}, velocity1, average = {np.mean(velo1): .04f}', alpha=.7)
    plt.plot(rm_time, velo2, label=f'{gif}, velocity2, average = {np.mean(velo2): .04f}', alpha=.7)

plt.xlabel('time (s)')
# plt.xlim(0,380)
plt.ylabel('Angular Velocity (deg/s)')
plt.legend(loc='upper right', bbox_to_anchor=(1.8, 0.8))
# plt.title('Ratchet Model')
plt.grid(True)
# plt.savefig(r'C:\Users\Nathan\OneDrive - nd.edu\Desktop\SANS Data\Experiments\PSI Cu2OSeO3 Corbino July 2023\Analysis\Field Sweep\Exported Figures\FieldSweepVelocities.png')
plt.show()